In [35]:
# session.write_pandas(df,table_name="HR_CHURN",auto_create_table=True).show()
# !pip install fosforml

In [3]:
import pandas as pd
df = pd.read_csv("HR_CHURN.csv",index_col='Unnamed: 0')

from snowflake.snowpark import Session
		
CONNECTION_PARAMETERS = {
"account": "RWB76633",
"user":"MAHESHG",
"password": "Jamesbond@123",
"role": "MAHESHG",
"database": "NVIDIA_DB",
"warehouse": "FOSFOR_WAREHOUSE",
"schema": "NVIDIA_SCHEMA", }
    
session = Session.builder.configs(CONNECTION_PARAMETERS).create()

In [4]:
from snowflake.snowpark.session import Session,col
import snowflake.snowpark.functions
from snowflake.ml.modeling.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
from snowflake.ml.modeling.preprocessing import LabelEncoder, StandardScaler
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.ensemble import GradientBoostingClassifier,RandomForestClassifier,RandomForestRegressor
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake
def apply_label_encoding(input_data_frame):
    for i in input_data_frame.dtypes:
        if i[1].find('string') >= 0:
            label_encoder = LabelEncoder(input_cols=i[0],output_cols=i[0],drop_input_cols=True)
            input_data_frame = label_encoder.fit(input_data_frame).transform(input_data_frame)
    return input_data_frame

In [5]:
snow_df = session.create_dataframe(df)
snow_df = snow_df.drop(['Unnamed: 0'])
input_data_frame, test_df = snow_df.randomSplit([0.75, 0.25])
label_encoder_column = []     
input_data_frame = apply_label_encoding(input_data_frame)
test_df = apply_label_encoding(test_df)
feature_cols = input_data_frame.columns
target_col = "SALARY"
feature_cols.remove(target_col)
OUTPUT_COLS = [target_col + '_PREDICTION']

In [ ]:
pipeline = GradientBoostingClassifier(input_cols=feature_cols, label_cols=target_col)
pipeline.fit(input_data_frame)
pred_snow_df = pipeline.predict(test_df)

Got error object of type 'NoneType' has no len() when trying to read default values from function: <function SnowparkModelTrainer._build_fit_wrapper_sproc.<locals>.fit_wrapper_function at 0x7f456e607af0>. Proceeding without creating optional arguments
The version of package 'snowflake-snowpark-python' in the local environment is 1.21.1, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


In [392]:
model_monitor_table = "MODEL_MONITORING"
model_alerts_table = "MODEL_ALERTS"
drift_output = {'metrics': [{'classification_quality_metrics': {'current': {'accuracy_score': 0.496894, 'f1_score': 0.4640358250210494, 'recall_score': 0.4968944099378882, 'precision_score': 0.47492237364556444, 'log_loss': 17.978025333432186, 'roc_auc_score': 0.6129413691930737, 'gini': 0.226}, 'reference': {'accuracy_score': 0.496894, 'f1_score': 0.4640358250210494, 'recall_score': 0.4968944099378882, 'precision_score': 0.47492237364556444, 'log_loss': 17.978025333432186, 'roc_auc_score': 0.6129413691930737, 'gini': 0.226}}}, {'class_representation': {'current': {'SALARY': {'"""PREDICT_PROBA_1.0"""': 1830, '"""PREDICT_PROBA_2.0"""': 1580, '"""PREDICT_PROBA_0.0"""': 293}, 'OUTPUT_SALARY': {'"""PREDICT_PROBA_1.0"""': 2568, '"""PREDICT_PROBA_2.0"""': 1095, '"""PREDICT_PROBA_0.0"""': 40}}, 'reference': {'SALARY': {'"""PREDICT_PROBA_1.0"""': 1830, '"""PREDICT_PROBA_2.0"""': 1580, '"""PREDICT_PROBA_0.0"""': 293}, 'OUTPUT_SALARY': {'"""PREDICT_PROBA_1.0"""': 2568, '"""PREDICT_PROBA_2.0"""': 1095, '"""PREDICT_PROBA_0.0"""': 40}}}}, {'confusion_matrix': {'current': [[12.0, 189.0, 92.0], [7.0, 1324.0, 499.0], [21.0, 1055.0, 504.0]], 'reference': [[12.0, 189.0, 92.0], [7.0, 1324.0, 499.0], [21.0, 1055.0, 504.0]]}}, {'quality_metrics_by_class': {'current': {'"""PREDICT_PROBA_0.0"""': {'precision_score': 0.3, 'recall_score': 0.040955631399317405, 'f1_score': 0.07207207207207207, 'accuracy_score': 0.9165541452876046}, '"""PREDICT_PROBA_1.0"""': {'precision_score': 0.5155763239875389, 'recall_score': 0.7234972677595628, 'f1_score': 0.6020918599363346, 'accuracy_score': 0.5274102079395085}, '"""PREDICT_PROBA_2.0"""': {'precision_score': 0.4602739726027397, 'recall_score': 0.3189873417721519, 'f1_score': 0.376822429906542, 'accuracy_score': 0.5498244666486632}}, 'reference': {'"""PREDICT_PROBA_0.0"""': {'precision_score': 0.3, 'recall_score': 0.040955631399317405, 'f1_score': 0.07207207207207207, 'accuracy_score': 0.9165541452876046}, '"""PREDICT_PROBA_1.0"""': {'precision_score': 0.5155763239875389, 'recall_score': 0.7234972677595628, 'f1_score': 0.6020918599363346, 'accuracy_score': 0.5274102079395085}, '"""PREDICT_PROBA_2.0"""': {'precision_score': 0.4602739726027397, 'recall_score': 0.3189873417721519, 'f1_score': 0.376822429906542, 'accuracy_score': 0.5498244666486632}}}}], 'metric_info': {'ACCURACY': 0.496894, 'F1_SCORE': 0.4640358250210494, 'PRECISION': 0.47492237364556444, 'RECALL': 0.4968944099378882}}

In [439]:
query = f"""
            INSERT INTO {model_monitor_table} 
                (object_id, model_name, version_name, object_type, object_data, created_by)
            SELECT 
                'ergerg' AS object_id,
                'testingmodel' AS model_name,
                'v2' AS version_name,
                'performancteste_drift_model' AS object_type,
                parse_json('{drift_ouput_json}') AS object_data,
                'mahesh' AS created_by;
"""

In [440]:
session.sql(query).collect()

[Row(number of rows inserted=1)]

In [7]:
execute_regression_drift(drift_configs,
                             sf_current,
                             sf_reference)

In [ ]:
pipeline = GradientBoostingClassifier(input_cols=feature_cols, label_cols=target_col)
pipeline.fit(input_data_frame)
pred_snow_df = pipeline.predict(test_df)

In [1]:
from fosforml import register_model

register_model(
  model_obj=pipeline,
  session=session,
  name="TestModelForDrifts",
  snowflake_df=pred_snow_df,
  dataset_name="HR_CHURN",
  dataset_source="Dataset",
  source="snowflake",
  description="This is a Snowflake model",
  flavour="snowflake",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

NameError: name 'pipeline' is not defined